# Check GPU availability

In [1]:

import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("GPU memory:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")
else:
    print("No GPU found - will use CPU")

PyTorch version: 2.8.0+cu128
CUDA available: True
GPU name: NVIDIA RTX 4000 Ada Generation
GPU memory: 21.01805056 GB


# Imports

In [5]:
import torch
import torch.nn as nn
import math

# Raw Implementation of Multi Head Attention



In [ ]:
import torch
import torch.nn as nn
import math

class MultiHeadAttention(nn.Module):
    """
    This is the standard multi-head attention class.
    """
    def __init__(self, d_model, num_heads, dropout=0.1):  # This runs when you create an object of this class
        super().__init__()  # This is used to call nn.module's init method which initializes the methods and attributes of the nn.module class
        assert d_model % num_heads == 0
        
        # We are storing all these so that they can be anywhere in the code
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
       
        # nn.Linear is PyTorch's fully connected (dense) layer that performs a linear transformation on the input.
        # It takes the input and multiplies it by a weight matrix and adds a bias term.
        # So it does a y=xw^T+b
        
        # So we need to create projections for Q, K, V (the parameters are input_dim, output_dim), so self.q_proj will create a weight matrix of size d_model x d_model,the weight initlization follows Xavier/Kaiming Initilication
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)

        # Post combination of all heads we need a final projection
        self.out_proj = nn.Linear(d_model, d_model)

        # Dropout helps us to randomly drop out some neurons to prevent overfitting
        self.dropout = nn.Dropout(dropout)
        
    # This is the method which runs when you call the model
    def forward(self, x):
        # This is tuple unpacking
        batch_size, seq_len, _ = x.size()  # Fixed: using _ instead of d_model to avoid shadowing

        # Now we need to project the input matrix into a different matrix
        # So we need to create projections for Q, K, V
        # Q: What am i looking for?
        # K: What do i contain?
        # V: What information do i have?

        Q = self.q_proj(x)  # Query = x@W_q^T + b_q  #This actually calls the forward method
        K = self.k_proj(x)  # Key = x@W_k^T + b_k
        V = self.v_proj(x)  # Value = x@W_v^T + b_v
        
        # Now we wish to split the query, key and value matrices into multiple attention heads so that we can perform parallel computations
        # Now we are reshaping the matrix to (batch_size, seq_len, num_heads, head_dim)
        Q = Q.view(batch_size, seq_len, self.num_heads, self.head_dim)
        K = K.view(batch_size, seq_len, self.num_heads, self.head_dim)
        V = V.view(batch_size, seq_len, self.num_heads, self.head_dim)

        # Now we need to transpose the matrix to put heads first
        # We are doing this since we want to compute attention for each head separately
        Q = Q.transpose(1, 2)
        K = K.transpose(1, 2)
        V = V.transpose(1, 2)

        # Compute attention scores
        # Scaling prevents softmax from saturating
        # scores[i,j]: how much token i should attend to token j high score means more attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        # Convert to probabilities
        attn_weights = torch.softmax(scores, dim=-1)

        # Apply dropout to the attention weights
        attn_weights = self.dropout(attn_weights)

        # We need to multiply with V
        # (batch_size, num_heads, seq_len, seq_len) * (batch_size, num_heads, seq_len, head_dim)
        # Here we are taking combination of information from all the heads weighted by attention
        output = torch.matmul(attn_weights, V)
        
        # We need to concatenate heads back
        # This is done to transpose the output and make it contiguous in memory (since a simple transpose is not contiguous)
        output = output.transpose(1, 2).contiguous()
        # This is concatenation of heads
        output = output.view(batch_size, seq_len, self.d_model)  # Fixed: batch -> batch_size, d_model -> self.d_model

        # Final Projection
        output = self.out_proj(output)

        return output


# Test the implementation
if __name__ == "__main__":
    # Create model
    model = MultiHeadAttention(d_model=512, num_heads=8, dropout=0.1)
    
    # Create input
    batch_size = 32
    seq_len = 10
    x = torch.randn(batch_size, seq_len, 512)
    
    # Forward pass
    output = model(x)
    
    print(f"Input shape:  {x.shape}")       # [32, 10, 512]
    print(f"Output shape: {output.shape}")   # [32, 10, 512]
    print("Multi-head attention works!")

Input shape:  torch.Size([32, 10, 512])
Output shape: torch.Size([32, 10, 512])
Multi-head attention works!


# Raw Implementation for GQA

In [4]:
import torch
import torch.nn as nn
import math

class GroupedQueryAttention(nn.Module):
    """
    This is the Grouped Query Attention (GQA) class.
    """ 
    def __init__(self, d_model, num_heads, num_kv_heads, dropout=0.1):  # This runs when you create an object of this class
        super().__init__()  # This is used to call nn.module's init method which initializes the methods and attributes of the nn.module class
        assert d_model % num_heads == 0
        assert num_heads % num_kv_heads == 0
        
        # We are storing all these so that they can be anywhere in the code
        self.d_model = d_model
        self.num_heads = num_heads
        self.num_kv_heads = num_kv_heads  # In GQA, we have num_kv_heads for K and V, and num_heads for Q. K and V heads are shared across Q head groups, so their initialization is different
        self.head_dim = d_model // num_heads
        self.group_size = num_heads // num_kv_heads  # How many Q heads will share one K, V head
       
        # nn.Linear is PyTorch's fully connected (dense) layer that performs a linear transformation on the input.
        # It takes the input and multiplies it by a weight matrix and adds a bias term.
        # So it does a y=xw^T+b
        
        # So we need to create projections for Q, K, V (the parameters are input_dim, output_dim), so self.q_proj will create a weight matrix of size d_model x d_model, the weight initialization follows Xavier/Kaiming initialization
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, num_kv_heads * self.head_dim)
        self.v_proj = nn.Linear(d_model, num_kv_heads * self.head_dim)

        # Post combination of all heads we need a final projection
        self.out_proj = nn.Linear(d_model, d_model)

        # Dropout helps us to randomly drop out some neurons to prevent overfitting
        self.dropout = nn.Dropout(dropout)
        
    # This is the method which runs when you call the model
    def forward(self, x):
        # This is tuple unpacking
        batch_size, seq_len, _ = x.size()  # Fixed: using _ instead of d_model to avoid shadowing

        # Now we need to project the input matrix into a different matrix
        # So we need to create projections for Q, K, V
        # Q: What am i looking for?
        # K: What do i contain?
        # V: What information do i have?

        Q = self.q_proj(x)  # Query = x@W_q^T + b_q  # This actually calls the forward method
        K = self.k_proj(x)  # Key = x@W_k^T + b_k
        V = self.v_proj(x)  # Value = x@W_v^T + b_v
        
        # Now we wish to split the query, key and value matrices into multiple attention heads so that we can perform parallel computations
        # Now we are reshaping the matrix to (batch_size, seq_len, num_heads, head_dim) for Q and (batch_size, seq_len, num_kv_heads, head_dim) for K and V
        Q = Q.view(batch_size, seq_len, self.num_heads, self.head_dim)
        K = K.view(batch_size, seq_len, self.num_kv_heads, self.head_dim)
        V = V.view(batch_size, seq_len, self.num_kv_heads, self.head_dim)

        # Now we need to transpose the matrix to put heads first
        # We are doing this since we want to compute attention for each head separately
        Q = Q.transpose(1, 2)
        K = K.transpose(1, 2)
        V = V.transpose(1, 2)

        # GQA: Repeat K and V heads to match Q's number of heads
        # repeat_interleave is used to repeat the elements of the tensor along a given dimension
        # Each K, V head is shared by group_size Q heads
        K = K.repeat_interleave(self.group_size, dim=1)  # (batch, num_heads, seq_len, head_dim)
        V = V.repeat_interleave(self.group_size, dim=1)  # (batch, num_heads, seq_len, head_dim)

        # Compute attention scores
        # Scaling prevents softmax from saturating
        # scores[i,j]: how much token i should attend to token j high score means more attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        # Convert to probabilities
        attn_weights = torch.softmax(scores, dim=-1)

        # Apply dropout to the attention weights
        attn_weights = self.dropout(attn_weights)

        # We need to multiply with V
        # (batch_size, num_heads, seq_len, seq_len) * (batch_size, num_heads, seq_len, head_dim)
        # Here we are taking combination of information from all the heads weighted by attention
        output = torch.matmul(attn_weights, V)
        
        # We need to concatenate heads back
        # This is done to transpose the output and make it contiguous in memory (since a simple transpose is not contiguous)
        output = output.transpose(1, 2).contiguous()
        # This is concatenation of heads
        output = output.view(batch_size, seq_len, self.d_model)  # Fixed: batch -> batch_size, d_model -> self.d_model

        # Final Projection
        output = self.out_proj(output)

        return output


# Test the implementation
if __name__ == "__main__":
    # Create GQA model
    # 8 Q heads, 2 K/V heads → 4 Q heads share each K/V head
    model = GroupedQueryAttention(d_model=512, num_heads=8, num_kv_heads=2, dropout=0.1)
    
    # Create input
    batch_size = 32
    seq_len = 10
    x = torch.randn(batch_size, seq_len, 512)
    
    # Forward pass
    output = model(x)
    
    print(f"Input shape:  {x.shape}")       # [32, 10, 512]
    print(f"Output shape: {output.shape}")   # [32, 10, 512]
    print("✅ Grouped Query Attention works!")

Input shape:  torch.Size([32, 10, 512])
Output shape: torch.Size([32, 10, 512])
✅ Grouped Query Attention works!


# MLA from scratch

In [6]:
import torch
import torch.nn as nn
import math

class MultiHeadLatentAttention(nn.Module):  # Inheritance
    def __init__(self, d_model, num_heads, d_latent, dropout=0.1):  # d_model is the dimension of the model, d_latent is the compression vector
        """
        Implementing Multi-Head Latent Attention from scratch
        """
        super().__init__()  # Call the parent class constructor
        assert d_model % num_heads == 0  # Assert is used to check if the condition is true
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_latent = d_latent  # Added the compression dimension
        self.head_dim = d_model // num_heads

        # Compress: Initialize the weight matrix for shared compression
        self.kv_compress = nn.Linear(d_model, d_latent)
        
        # Expand latent to K and V for all heads
        self.k_proj = nn.Linear(d_latent, d_model)
        self.v_proj = nn.Linear(d_latent, d_model)

        # NOTE: Q remains the same, there is no compression in Q
        self.q_proj = nn.Linear(d_model, d_model)
        
        # Output projection (This is after attention we need to concatenate all heads)
        self.out_proj = nn.Linear(d_model, d_model)

        # Create a dropout layer to prevent overfitting
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Unpack the input
        batch_size, seq_len, _ = x.size()  # Fixed: using _ instead of d_model to avoid shadowing

        # Project Q (NOTE: Projection just means x@W+b)
        Q = self.q_proj(x)

        # COMPRESS: Compress input to latent space (THE KEY STEP!)
        # Here you compress the input into a smaller latent vector
        kv_latent = self.kv_compress(x)

        # Now we need to expand latent into K and V
        K = self.k_proj(kv_latent)
        V = self.v_proj(kv_latent)
        
        # Here we are reshaping so that each head can compute attention independently and in parallel
        Q = Q.view(batch_size, seq_len, self.num_heads, self.head_dim)
        K = K.view(batch_size, seq_len, self.num_heads, self.head_dim)
        V = V.view(batch_size, seq_len, self.num_heads, self.head_dim)

        # Transpose to put heads first
        Q = Q.transpose(1, 2)
        K = K.transpose(1, 2)
        V = V.transpose(1, 2)

        # Compute the attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        
        # Convert scores into attention weights
        attn_weights = torch.softmax(scores, dim=-1)
        
        # Apply dropout
        attn_weights = self.dropout(attn_weights)
        
        # Multiply with V
        output = torch.matmul(attn_weights, V)
        
        # Transpose back
        output = output.transpose(1, 2).contiguous()
       
        # Concat all heads
        output = output.view(batch_size, seq_len, self.d_model)

        # Final Output
        output = self.out_proj(output)
         
        return output


# Test the implementation
if __name__ == "__main__":
    # Create model
    # d_model=512, d_latent=128 means we compress from 512 → 128 → 512
    model = MultiHeadLatentAttention(d_model=512, num_heads=8, d_latent=128, dropout=0.1)
    
    # Create input
    batch_size = 32
    seq_len = 10
    x = torch.randn(batch_size, seq_len, 512)
    
    # Forward pass
    output = model(x)
    
    print(f"Input shape:         {x.shape}")       # [32, 10, 512]
    print(f"Output shape:        {output.shape}")   # [32, 10, 512]
    


Input shape:         torch.Size([32, 10, 512])
Output shape:        torch.Size([32, 10, 512])


# BenchMarking

In [8]:
import torch
import torch.nn as nn
import time
# from attention import MultiHeadAttention, GroupedQueryAttention, MultiHeadLatentAttention

In [9]:
# Configuration
d_model = 512
num_heads = 8
num_kv_heads = 2  # For GQA
d_latent = 128    # For MLA
seq_len = 1024
batch_size = 2

In [10]:
# Create models
mha = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
gqa = GroupedQueryAttention(d_model=d_model, num_heads=num_heads, num_kv_heads=num_kv_heads)
mla = MultiHeadLatentAttention(d_model=d_model, num_heads=num_heads, d_latent=d_latent)

print("✅ All models created!")

✅ All models created!


In [12]:
# Check if GPU is available and move models
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

mha = mha.to(device)
gqa = gqa.to(device)
mla = mla.to(device)

print("✅ Models moved to device!")

Using device: cuda
✅ Models moved to device!


In [13]:
# Create random input tensor
x = torch.randn(batch_size, seq_len, d_model).to(device)
print(f"Input shape: {x.shape}")
print("✅ Input created!")


Input shape: torch.Size([2, 1024, 512])
✅ Input created!


In [16]:
def calculate_kv_cache_size(model, seq_len, batch_size=1):
    """Calculate theoretical KV cache size in MB"""
    
    if isinstance(model, MultiHeadAttention):
        # MHA: num_heads K + num_heads V
        kv_heads = model.num_heads * 2  # K and V
    elif isinstance(model, GroupedQueryAttention):
        # GQA: num_kv_heads K + num_kv_heads V
        kv_heads = model.num_kv_heads * 2  # K and V
    elif isinstance(model, MultiHeadLatentAttention):
        # MLA: compressed latent for K and V
        # Cache size = 2 * d_latent (for K and V latent)
        bytes_per_element = 4  # float32
        cache_size = 2 * batch_size * seq_len * model.d_latent * bytes_per_element
        return cache_size / (1024 ** 2)  # Convert to MB
    
    # For MHA and GQA
    head_dim = model.head_dim
    bytes_per_element = 4  # float32
    cache_size = kv_heads * batch_size * seq_len * head_dim * bytes_per_element
    return cache_size / (1024 ** 2)  # Convert to MB

print("\n🔍 KV Cache Size (what matters for inference!):\n")

mha_cache = calculate_kv_cache_size(mha, seq_len)
gqa_cache = calculate_kv_cache_size(gqa, seq_len)
mla_cache = calculate_kv_cache_size(mla, seq_len)

print(f"MHA KV cache: {mha_cache:.2f} MB")
print(f"GQA KV cache: {gqa_cache:.2f} MB")
print(f"MLA KV cache: {mla_cache:.2f} MB")

print(f"\n🎉 Real Savings:")
print(f"GQA vs MHA: {mha_cache/gqa_cache:.2f}x smaller")
print(f"MLA vs MHA: {mha_cache/mla_cache:.2f}x smaller")


🔍 KV Cache Size (what matters for inference!):

MHA KV cache: 4.00 MB
GQA KV cache: 1.00 MB
MLA KV cache: 1.00 MB

🎉 Real Savings:
GQA vs MHA: 4.00x smaller
MLA vs MHA: 4.00x smaller
